# YOLOv5 Object Detection Model with OpenCV

### This is designed to detect objects in an uploaded YouTube video.

### First, import necessary libraries

In [1]:
import torch
import numpy as np
import cv2
import pafy
from time import time

### Next, implement an object detection class

In [4]:
class ObjectDetection:
    #Class implements Yolo5 model to make inferences on a youtube video using OpenCV.
    
    def __init__(self, url, out_file):
        #Initializes the class with YouTube URL and output file.
        #:param url: The prediction we make has to be on a YouTube URL.
        #:param out_file: A valid output file name.
        
        self._URL = url
        self.model = self.load_model()
        self.classes = self.model.names
        self.out_file = out_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu' #use cpu if pytorch gpu support isn't there
        print("\n\nDevice Used: ", self.device)
        
    def get_video_from_url(self):
        #Creates a new video streaming object to extract frames from to make predictions on.
        # returns opencv2 video capture object with lowest quality frame available for video.
        
        play = pafy.new(self._URL).streams[-1]
        assert play is not None #otherwise we can't load in the URL
        return cv2.VideoCapture(play.url)
    
    def load_model(self):
        
        #Loads YOLOv5 model from PyTorch hub.
        #returns trained PyTorch model.
        
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained = True)
        return model
    
    def score_frame(self, frame):
        
        #Takes a single frame as input, and scores the frame using YOLOv5 model.
        #:param frame: input frame in numpy/list/tuple format
        #returns Labels and Coordinates of objects detected by model in the frame.
        
        self.model.to(self.device)
        frame = [frame]
        results = self.model(frame)
        
        labels, cord = results.xyxyn[0][:,-1], results.xyxyn[0][:,:-1] #xyxyn represents the coordinates and label values
        return labels, cord
    
    
    def class_to_label(self, x):
        #For a given label value, return corresponding string label
        #:param: numeric label
        #return corresponding string label
        
        return self.classes[int(x)]
    
    def plot_boxes(self, results, frame):
        
        #Takes a frame and its results as an input, and plots the bounding boxes and lavel onto the frame.
        #:param results: contains labels ad coordinates predicted by model on the given frame
        #return frame with bouding boxes and labels plotted on it.
        
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0,255,0)
                cv2.rectangle(frame, (x1,y1), (x2,y2), bgr, 2)
                cv2.putText(frame, self.class_to_label(labels[i]), (x1,y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
        
        return frame
    
    
    def __call__(self):
        
        #This function is called when the class is executed, it runs the loop to read the video frame by frame and writes the output to a new fie
        
        player = self.get_video_from_url()
        assert player.isOpened()
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 20, (x_shape, y_shape))
        
        while True: #while there are frames in video, call these functions
            start_time = time()
            ret, frame = player.read()
            if not ret:
                break
            
            results = self.score_frame(frame)
            frame = self.plot_boxes(results, frame)
            end_time = time()
            fps = 1/np.round(end_time - start_time, 3)
            print(f"Frames Per Second : {fps}")
            out.write(frame)
        

#Create a new object and execute
#detection = ObjectDetection("insert youtube URL here", ".avi file here")
#detection()
        